In [3]:
%matplotlib notebook
import matplotlib.pyplot as plt 
import numpy as np
import torch

import nmf.mult
import nmf.pgrad
import nmf.nesterov

import nmf_torch.mult
import nmf_torch.pgrad
import nmf_torch.nesterov
import nmf_torch.norms

from theory.represent import rescale_WH

from read_data.reading import read_indian_pines, images_matrix_grid,\
                                roll_images, unroll_images 

from performance.performance_eval_func import get_random_lowrank_matrix, get_time_ratio,\
                      compare_performance, plot_performance_dict,\
                      torch_algo_wrapper,\
                      plot_ratios_gpu_algo, plot_ratios_cpu_gpu, plot_ratios_cpu_algo,\
                      errors_at_time_t_over_inner_dim
                                                

In [ ]:
import importlib, performance.performance_eval_func
importlib.reload(performance.performance_eval_func)

In [4]:
from matplotlib.gridspec import GridSpec
from matplotlib.image import AxesImage
import matplotlib.widgets as widgets


class InteractiveLandPlot:
    def __init__(self, gt_data, H, ori_shape):
        gs = GridSpec(2, 4, width_ratios=[2, 10, 10, 2], height_ratios=[10, 1])

        self.maps_fig = plt.figure(figsize=(5, 4))
        self.maps_ax = plt.subplot(gs[0, 2:])
        self.btn_less_ax = plt.subplot(gs[1, 0])
        self.slider_ax = plt.subplot(gs[1, 1:3])
        self.btn_more_ax = plt.subplot(gs[1, 3])
        self.maps_ax.set_title("Where the ith type appears")

        self.gt_ax = plt.subplot(gs[0, :2], sharex=self.maps_ax, 
                                  sharey=self.maps_ax)
         
        self.gt_ax.imshow(gt_data, cmap="nipy_spectral")
        self.gt_ax.set_title("Ground truth")      
            
        self.maps = roll_images(H, ori_shape)

        self.slider = widgets.Slider(self.slider_ax,
                                     label="",
                                     valmin=0,
                                     valmax=len(self.maps) - 1,
                                     valfmt='%0.0f',
                                     valstep=1)
        self.slider.on_changed(self.on_slider_move)

        self.btn_less = widgets.Button(self.btn_less_ax, "<<")
        self.btn_less.on_clicked(lambda e: self.set_slider_val(self.slider.val - 1))

        self.btn_more = widgets.Button(self.btn_more_ax, ">>")
        self.btn_more.on_clicked(lambda e: self.set_slider_val(self.slider.val + 1))
        
        self.on_slider_move(0)

    def set_slider_val(self, val):
        val = np.clip(val, self.slider.valmin, self.slider.valmax)
        self.slider.set_val(val)

    def on_slider_move(self, val):
        self.plot_ith_term(int(val))

    def plot_ith_term(self, i):
        for artist in self.maps_ax.get_children():
            if isinstance(artist, AxesImage): artist.remove()

        self.maps_ax.imshow(self.maps[i], zorder=0)
        self.maps_fig.canvas.draw()

In [ ]:
algo_dict_to_test = {
    "mult": nmf.mult.factorise_Fnorm,
    "pgrad": nmf.pgrad.factorise_Fnorm_subproblems,
    "nesterov": nmf.nesterov.factorise_Fnorm,

    "mult_torch": torch_algo_wrapper(nmf_torch.mult.factorise_Fnorm, 
                                     device="cuda"),
    "pgrad_torch": torch_algo_wrapper(nmf_torch.pgrad.factorise_Fnorm_subproblems, 
                                      device="cuda"),
    "nesterov_torch": torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm, 
                                        device="cuda")
}

In [ ]:
indian_pines_data = read_indian_pines("data/indian_pines/images/")

In [ ]:
indian_pines_data.keys()

In [ ]:
site3_im = indian_pines_data["site3_im"]
ns_line_im = indian_pines_data["ns_line_im"]
site3_gt_im = indian_pines_data["site3_gt_im"]
ns_line_gt_im = indian_pines_data["ns_line_gt_im"]

plt.figure()
plt.subplot(121)
plt.imshow(ns_line_gt_im, cmap="nipy_spectral")
plt.subplot(122)
plt.imshow(site3_gt_im, cmap="nipy_spectral")

In [ ]:
print(len(set(site3_gt_im.ravel())))
print(len(set(ns_line_gt_im.ravel())))

In [ ]:
site3_unrolled_data, site3_ori_shape = unroll_images(site3_im)
ns_line_unrolled_data, ns_line_ori_shape = unroll_images(ns_line_im)

# Site 3

In [ ]:
errors_result_site3 = errors_at_time_t_over_inner_dim(
                            site3_unrolled_data, range(5, 35, 2),
                            30, {"nesterov_torch":algo_dict_to_test["nesterov_torch"]})

In [ ]:
f, ax = plt.subplots()
ax.set_title("site_3")
plot_dict(errors_result_site3, ax, log=False)

In [ ]:
shape = site3_unrolled_data.shape
inner_dim=24

W_init = np.random.rand(shape[0], inner_dim)
H_init = np.random.rand(inner_dim, shape[1])

site3_errors = compare_performance(V=site3_unrolled_data, 
                                  inner_dim=inner_dim, time_limit=240,
                                  W_init=W_init, H_init=H_init, 
                                  algo_dict_to_test=algo_dict_to_test)

NameError: name 'site3_errors' is not defined

In [ ]:
f, axs = plt.subplots(2, 2)
f.suptitle("site3{} factorisation with inner_dim = {}".format(site3_unrolled_data.shape, inner_dim))
axs[0, 0].set_title("log(error)")
plot_performance_dict(errors, axs[0, 0], log=True)

plot_ratios_cpu_gpu(site3_errors, axs[0, 1])
plot_ratios_cpu_algo(site3_errors, axs[1, 0])
plot_ratios_gpu_algo(site3_errors, axs[1, 1])

In [ ]:
W_site3, H_site3 = torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm)(
    V=site3_unrolled_data, inner_dim=25, 
    time_limit=60, epsilon=1e-6, max_steps=10000)

In [ ]:
InteractiveLandPlot(site3_gt_im, H_site3, site3_ori_shape)

# N-S Line

In [ ]:
errors_result_ns_line = errors_at_time_t_over_inner_dim(
                            ns_line_unrolled_data, range(50, 80, 2),
                            30, {"nesterov_torch":algo_dict_to_test["nesterov_torch"]})

In [ ]:
f, ax = plt.subplots()
ax.set_title("ns_line")
plot_dict(errors_result_ns_line, ax, log=False)

In [ ]:
shape = ns_line_unrolled_data.shape
inner_dim=65

W_init = np.random.rand(shape[0], inner_dim).astype(np.float32)
H_init = np.random.rand(inner_dim, shape[1]).astype(np.float32)

ns_line_errors = compare_performance(V=ns_line_unrolled_data.astype(np.float32), 
                                  inner_dim=inner_dim, time_limit=600,
                                  W_init=W_init, H_init=H_init, 
                                  algo_dict_to_test=algo_dict_to_test)

In [ ]:
f, axs = plt.subplots(2, 2)
f.suptitle("ns_line{} factorisation with inner_dim = {}".format(ns_line_unrolled_data.shape, inner_dim))
axs[0, 0].set_title("log(error)")

plot_performance_dict(ns_line_errors, axs[0, 0])
plot_ratios_cpu_gpu(ns_line_errors, axs[0, 1])
plot_ratios_cpu_algo(ns_line_errors, axs[1, 0])
plot_ratios_gpu_algo(ns_line_errors, axs[1, 1])

In [ ]:
W_ns_line, H_ns_line = torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm)(
    V=ns_line_unrolled_data.astype(np.float32), inner_dim=70, 
    time_limit=60, epsilon=1e-6, max_steps=10000)

In [ ]:
InteractiveLandPlot(ns_line_gt_im, H_ns_line, ns_line_ori_shape)